In [ ]:
import numpy as np
import cv2
import cvzone
from ultralytics import YOLO
import pygame
import os
path = r"D:\Computer Vision\Real-Time Smoke and Gas Leak Detection"

os.chdir(path = path)


# Model 
model = YOLO(r"best.pt")


# Data
df = open(r"coco1.txt" , "r")
classes = df.read().split("\n")


cap = cv2.VideoCapture(r"Fire.mp4")


pygame.init()
pygame.mixer.music.load("alarm.wav")


while True :
    ret , frame = cap.read()
    if ret == False :
        print("Error , Frame Not Found.")
        break
    
    results = model.predict(frame)
    
    lis = []
    for result in results :
        for box in result.boxes :
            x1 , y1 , x2 , y2 = map(int , box.xyxy[0])
            if int(box.cls[0]) < len(classes) :
                name = classes[int(box.cls[0])]
                lis.append([x1 , y1 , x2 , y2])
                ww = x2-x1
                hh = y2-y1

                cx = int((x1+x2) // 2)
                cy = int((y1+y2) // 2)

                
        cvzone.putTextRect(frame, f" {name}", (x1 + 10, y1 - 10), scale=1, thickness=1, colorR=(0, 255, 0), colorT=(0, 0, 0))
        cvzone.cornerRect(frame, (x1, y1, ww, hh), l=5, t=10, rt=2, colorR=(0, 255, 0), colorC=(255, 0, 255))


        if "Fire" in name:
            cvzone.putTextRect(frame, f" {name}", (x1 + 10, y1 - 10), scale=1, thickness=1, colorR=(0, 0, 255), colorT=(0, 0, 0))
            cvzone.cornerRect(frame, (x1, y1, ww, hh), l=10, t=10, rt=2, colorR=(0, 0, 255), colorC=(0, 0, 255))

            if not pygame.mixer.music.get_busy():
                    pygame.mixer.music.play()


                    
    cv2.imshow("Real-Time Smoke and Gas Leak Detection", frame)

    if cv2.waitKey(1) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
